In [32]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [33]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.0.0-alpha0


In [34]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [35]:
# dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
# dataset_path

dataset_path = 'csv/ABBV.timeseries.csv'
dataset_path

'csv/ABBV.timeseries.csv'

In [36]:
# column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
#                 'Acceleration', 'Model Year', 'Origin']
# raw_dataset = pd.read_csv(dataset_path, names=column_names,
#                       na_values = "?", comment='\t',
#                       sep=" ", skipinitialspace=True)

column_names = ['date','open','high','low','close','volume','adjclose']
raw_dataset = pd.read_csv(dataset_path, names=column_names)

dataset = raw_dataset.copy()
dataset.tail()

,date,open,high,low,close,volume,adjclose
1757,2013-01-08,34.290000915527344,34.63999938964844,33.36000061035156,33.709999084472656,17863300,25.726606369018555
1758,2013-01-07,34.150001525878906,35.45000076293945,34.150001525878906,34.459999084472656,17897100,26.298986434936523
1759,2013-01-04,34.619998931884766,34.88999938964844,34.25,34.38999938964844,21372100,26.245569229125977
1760,2013-01-03,35,35,34.15999984741211,34.83000183105469,16739300,26.58136749267578
1761,2013-01-02,34.91999816894531,35.400001525878906,34.099998474121094,35.119998931884766,13767900,26.80267906188965


In [37]:
dataset.isna().sum()

date        0
open        0
high        0
low         0
close       0
volume      0
adjclose    0
dtype: int64

In [38]:
# dataset = dataset.dropna()

In [39]:
# dataset['date'] = dataset['date'].map(lambda x: {1: 'USA', 2: 'Europe', 3: 'Japan'}.get(x))
# dataset = dataset.drop(["date","close"], axis = 1)
dataset = dataset.drop(["adjclose"], axis = 1)
dataset.head()

,date,open,high,low,close,volume
0,date,open,high,low,close,volume
1,2019-12-30,89.22000122070312,89.7300033569336,88.16000366210938,88.5199966430664,7014800
2,2019-12-27,89.9800033569336,90.27999877929688,88.83000183105469,89.19999694824219,4087800
3,2019-12-26,90.19999694824219,90.72000122070312,89.44999694824219,89.83000183105469,4066700
4,2019-12-24,90.4000015258789,90.4000015258789,89.43000030517578,89.8499984741211,1798800


In [40]:
# dataset = pd.get_dummies(dataset)
dataset.tail()

,date,open,high,low,close,volume
1757,2013-01-08,34.290000915527344,34.63999938964844,33.36000061035156,33.709999084472656,17863300
1758,2013-01-07,34.150001525878906,35.45000076293945,34.150001525878906,34.459999084472656,17897100
1759,2013-01-04,34.619998931884766,34.88999938964844,34.25,34.38999938964844,21372100
1760,2013-01-03,35,35,34.15999984741211,34.83000183105469,16739300
1761,2013-01-02,34.91999816894531,35.400001525878906,34.099998474121094,35.119998931884766,13767900


In [41]:
train_dataset = dataset.sample(frac=0.7,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
# sns.pairplot(train_dataset[['open','high','low','volume','adjclose']], diag_kind="kde")
sns.pairplot(train_dataset[['open','high','low','close']], kind="reg")

In [28]:
train_stats = train_dataset.describe()
# train_stats.pop("adjclose")
train_stats.pop("close")
train_stats = train_stats.transpose()
train_stats

,count,unique,top,freq
open,1233,1093,94,4
high,1233,1100,89.98999786376953,3
low,1233,1111,63.5,4


In [29]:
# train_labels = train_dataset.pop('adjclose')
# test_labels = test_dataset.pop('adjclose')

train_labels = train_dataset.pop('close')
test_labels = test_dataset.pop('close')

In [30]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

KeyError: 'mean'

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

In [ ]:
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [adjclose]')
plt.ylabel('MAE [close]')

In [ ]:
plotter.plot({'Basic': history}, metric = "mse")
plt.ylim([0, 20])
plt.ylabel('MSE [adjclose^2]')
plt.ylabel('MSE [close^2]')

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

In [ ]:
plotter.plot({'Early Stopping': early_history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [adjclose]')
plt.ylabel('MAE [close]')

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} close".format(mae))
print("Testing set Mean Abs Error: {:5.2f} adjclose".format(mae))

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [close]')
plt.ylabel('Predictions [close]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [close]")
_ = plt.ylabel("Count")